## Chat sobre documentos PDF : RAG con LangChain, Ollama, y FAISS Vector Store

In [1]:
# pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

In [2]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

### Document Loader

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

# loader = PyMuPDFLoader("./dgi-dataset/*.pdf")
# docs = loader.load()
# load multiple pdfs
# poner directorio donde se encuentran los pdfs
pdf_directory = ".\dgi-dataset"
documents = []

for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        loader = PyMuPDFLoader(os.path.join(pdf_directory, filename))
        documents.extend(loader.load())

In [4]:
documents[0]

Document(metadata={'source': '.\\dgi-dataset\\798+12+web+06+024 (2).pdf', 'file_path': '.\\dgi-dataset\\798+12+web+06+024 (2).pdf', 'page': 0, 'total_pages': 25, 'format': 'PDF 1.7', 'title': 'Microsoft Word - 27F1BEE0.doc', 'author': 'Ileana Olivera', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240809093858-03'00'", 'modDate': "D:20240809095909-03'00'", 'trapped': ''}, page_content=' \nActualización Junio 2024 \n \nRESOLUCIÓN D.G.I. Nº 798/2012 \n \n \nDOCUMENTACIÓN FISCAL ELECTRÓNICA –SE ESTABLECEN LAS CONDICIONES QUE REGULEN SU \nRÉGIMEN. \n \n \nMontevideo, 8 de mayo de 2012 \n \nVisto: la Ley Nº 18.600 de 21 de setiembre de 2009 y el Decreto Nº 36/012 de 8 de febrero de \n2012. \n \nResultando: I) Que la citada norma legal reconoce la validez y la eficacia jurídica de los \ndocumentos electrónicos y la firma electrónica; \n \nII) Que el mencionado decreto comete a la Dirección General Impositiva el dictado de normas \nc

In [5]:
import os

pdfs = []
for root, dirs, files in os.walk('dgi-dataset'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

In [6]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)

In [7]:
len(docs)

305

### Document Chunking

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)

In [9]:
len(docs), len(chunks)

(305, 830)

In [10]:
len(docs[0].page_content), len(chunks[0].page_content)

(2451, 922)

In [13]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

len(encoding.encode(docs[0].page_content)), len(encoding.encode(chunks[0].page_content))

(632, 245)

### Document Vector Embedding

In [14]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [15]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

single_vector = embeddings.embed_query("this is some text data")


In [16]:
len(single_vector)

768

In [17]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [18]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [19]:
len(chunks)

830

In [20]:
ids = vector_store.add_documents(documents=chunks)

In [21]:
vector_store.index_to_docstore_id
len(ids)

830

In [22]:
# # store vector database
db_name = "db_name"
vector_store.save_local(db_name)

# # # load vector database
new_vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)
len(new_vector_store.index_to_docstore_id)

830

### Retreival

In [24]:
question = "¿Cuándo debo hacer una nota de crédito?"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")

son: 
 e-Factura. 
 
 Nota de crédito de e- Factura. 
 
Para operaciones entre contribuyentes. 
 Nota de débito de e- Factura. 
 
 
(B2B) 
 
 e-Ticket. 
 Nota de crédito de e-Ticket. 
 
Sólo para Consumo Final. 
 Nota de débito de e-Ticket.  
 
 
(B2C) 
 
 e- Factura de Exportación 
 Nota de crédito de e- Factura de Exportación     Sólo para exportaciones. 
 Nota de débito de e-Factura de Exportación



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia



Cuenta Ajena, e-Boleta de entrada, Nota de Crédito de e-Boleta de entrada y Nota de 
Débito de e-Boleta de entrada>  
 
de conting

In [25]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs = {'k': 3, 
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})

In [26]:
docs = retriever.invoke(question)

for doc in docs:
     print(doc.page_content)
     print("\n\n")


son: 
 e-Factura. 
 
 Nota de crédito de e- Factura. 
 
Para operaciones entre contribuyentes. 
 Nota de débito de e- Factura. 
 
 
(B2B) 
 
 e-Ticket. 
 Nota de crédito de e-Ticket. 
 
Sólo para Consumo Final. 
 Nota de débito de e-Ticket.  
 
 
(B2C) 
 
 e- Factura de Exportación 
 Nota de crédito de e- Factura de Exportación     Sólo para exportaciones. 
 Nota de débito de e-Factura de Exportación



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia





In [27]:


question = "¿Cuándo debo hacer una nota de crédito?"
docs = retriever.invoke(question)


### RAG con LLAMA 3.2 sobre OLLAMA

In [28]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_ollama import ChatOllama

from langchain_openai import ChatOpenAI

In [31]:
model = ChatOllama(model="llama3.2:1b", base_url="http://localhost:11434")

#model = ChatOpenAI(model="gpt-4o")

model.invoke("hi")

AIMessage(content='How can I help you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-27T21:06:37.144101Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3459603500, 'load_duration': 2440131300, 'prompt_eval_count': 26, 'prompt_eval_duration': 510000000, 'eval_count': 8, 'eval_duration': 501000000}, id='run-da7a2edf-d5c5-4bb3-b01c-5539a7a4db20-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [32]:
prompt = hub.pull("rlm/rag-prompt")

In [33]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know. 
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [34]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# print(format_docs(docs))

In [35]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [36]:
question = "¿Cuándo debo hacer una nota de crédito?"
output = rag_chain.invoke(question)
print(output)


En resumen, la nota de crédito se utiliza para operaciones entre contribuyentes y es solo para consumo final. La notación correcta para obtener una nota de crédito sería:

* Nota de débito de e- Factura (B2C)
* Nota de crédito de e-Factura (B2B)


In [37]:
question = "¿Qué es un RDC?"
output = rag_chain.invoke(question)
print(output)

* El Reporte Diario Consolidado (RDC) es un archivo electrónico que contiene información sobre comprobantes fiscales electrónicos emitidos por cada tipo de CFE/CFC.
 + La información incluye datos de la numeración utilizada, fecha y sucursal.


In [38]:
question = "¿Qué es un CFE?"
output = rag_chain.invoke(question)
print(output)

Aquí te presento la información relevante sobre "CFE" que puedo proporcionar:

* Un CFE (Comprobante Fiscal de Contingencia) es un tipo de comprobante papel emitido por las autoridades competentes en situaciones de contingencia.
* Está definido por el DGI (Dirección General de Aduanas y Tributación) y asignado un código para cada tipo de CFE, que deberán ser de uso obligatorio para todos los contribuyentes.
* Los comprobantes papel utilizados en situaciones de contingencia deben estar acompañados de correcciones y/o ajustes al documento original, así como notas de crédito y débito.
* El conjunto mínimo obligatorio de comprobantes fiscales electrónicos con los que se puede ingresar en el sistema está compuesto por e-Factura y e-Ticket, junto con sus correspondientes notas de crédito y débito para las correcciones y/o ajustes al documento original.


### Cambiamos el modelo a OpenAI y realizamos las mismas consultas

In [40]:
model = ChatOpenAI(model="gpt-4o")

In [42]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
question = "¿Cuándo debo hacer una nota de crédito?"
output = rag_chain.invoke(question)
print(output)

- Debes hacer una nota de crédito de e-Factura para operaciones entre contribuyentes.
- Debes hacer una nota de crédito de e-Ticket sólo para consumo final.
- Debes hacer una nota de crédito de e-Factura de Exportación sólo para exportaciones.


In [44]:
question = "¿Qué es un RDC?"
output = rag_chain.invoke(question)
print(output)

- Un RDC es un "Reporte Diario Consolidado".
- Contiene información consolidada de todos los comprobantes fiscales electrónicos y documentos de contingencia emitidos durante un día.
- El informe se genera por cada tipo de comprobante fiscal certificado, fecha de comprobante y sucursal.
- La búsqueda y consulta de RDC permite acceder a los reportes generados para las empresas en la plataforma.
- Los estados del RDC pueden ser: "Aceptado" (confirmado por DGI), "Almacenado" (generado pero no enviado a DGI), y "Rechazado" (enviado y rechazado por DGI).


In [45]:
question = "¿Qué es un CFE?"
output = rag_chain.invoke(question)
print(output)

- Un CFE es un Comprobante Fiscal Electrónico.
- Son documentos fiscales electrónicos utilizados para registrar transacciones de manera digital.
- La Dirección General Impositiva (DGI) asigna códigos a cada tipo de CFE para su identificación.
- Los CFE pueden incluir e-Factura, e-Ticket, y sus correspondientes notas de crédito y débito.
- En situaciones donde no se pueden emitir CFE, se utilizan Comprobantes Fiscales de Contingencia (CFC).
- Los estados de un CFE pueden variar desde "CFE Almacenado" hasta "CFE Confirmado" o "CFE Rechazado".


### Se comprueba que los tiempos y la calidad de respuestas son mucho mejores con OPENAI que corriendo llama3.2:1b en la máquina local.

### Gradio Interface

In [46]:
import gradio as gr 

In [47]:
def chatbot(question):
    															  
    docs = retriever.invoke(question)

    rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser())

    output = rag_chain.invoke(question)

    return output

    # streams option
    # result = ""
    # for chunk in stream:
    #     result += chunk.choices[0].delta.content or ""
    #     yield result


In [48]:
view = gr.Interface(
    fn=chatbot,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=20)],
    flagging_mode="never"
)

gr.themes.Ocean() 

view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://859842b6cf75693387.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
